In [ ]:
pip install transformers

In [ ]:
pip install transformers[torch]

In [ ]:
pip install datasets transformers pandas

In [ ]:
pip install sentencepiece

In [ ]:
pip install evaluate

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset, DatasetDict
import evaluate
import json

In [8]:
#laod the dataset 
with open('documents_with_descriptions.json', 'r') as file:
    raw_data = json.load(file)

In [ ]:
raw_data

In [10]:
data = {
    "input": [
         f"Create a CWL pipeline for {entry['metadata'].get('description', 'an unspecified task')} using {entry['metadata'].get('baseCommand', 'a specific tool')}"
        for entry in raw_data
    ],
    "target_text": [entry['page_content'] for entry in raw_data]
} 

In [ ]:
data

In [11]:
model_name = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [12]:
def generate_alternative_description(desc):
    # Preprocess the CWL content
    cwl_summary_prompt = f"Create a cwl pipeline:\n{desc}"
    
    # Encode the CWL content
    input_ids = tokenizer.encode(cwl_summary_prompt, return_tensors="pt", max_length=512, truncation=True)

    # Generate the description
    output = model.generate(
        input_ids,
        max_length=150,
        num_beams=5,
        no_repeat_ngram_size=2,
        num_return_sequences=1,
        temperature=0.7,  # Balance randomness and coherence
        top_k=50,
        top_p=0.9
    )
    
    # Decode the output
    alternative_description = tokenizer.decode(output[0], skip_special_tokens=True)
    
    return alternative_description

In [13]:
expanded_data = []

# Loop through your raw data and generate multiple descriptions for each target_text
n = len(raw_data)
print(n)
for entry in raw_data:
    target_text = entry['page_content']
    descriptions = generate_alternative_description(entry['metadata'].get('description'))  # Generate 5 variations per target_text

    # Create a list of (input, target_text) pairs for each target_text
    group = {"input": descriptions, "target_text": target_text}
    
    # Append the group to the expanded data
    expanded_data.append(group)
    n = n -1 
    print(n)
    

# Example output: View the first group
print(expanded_data)  # Print the first group

785


/opt/conda/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


784
783
782
781
780
779
778
777
776
775
774
773
772
771
770
769
768
767
766
765
764
763
762
761
760
759
758
757
756
755
754
753
752
751
750
749
748
747
746
745
744
743
742
741
740
739
738
737
736
735
734
733
732
731
730
729
728
727
726
725
724
723
722
721
720
719
718
717
716
715
714
713
712
711
710
709
708
707
706
705
704
703
702
701
700
699
698
697
696
695
694
693
692
691
690
689
688
687
686
685
684
683
682
681
680
679
678
677
676
675
674
673
672
671
670
669
668
667
666
665
664
663
662
661
660
659
658
657
656
655
654
653
652
651
650
649
648
647
646
645
644
643
642
641
640
639
638
637
636
635
634
633
632
631
630
629
628
627
626
625
624
623
622
621
620
619
618
617
616
615
614
613
612
611
610
609
608
607
606
605
604
603
602
601
600
599
598
597
596
595
594
593
592
591
590
589
588
587
586
585
584
583
582
581
580
579
578
577
576
575
574
573
572
571
570
569
568
567
566
565
564
563
562
561
560
559
558
557
556
555
554
553
552
551
550
549
548
547
546
545
544
543
542
541
540
539
538
537
536
535


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
print(expanded_data[0]['input'])

In [ ]:
print(data['input'])

In [14]:
print(len(expanded_data))
for i in range(len(expanded_data)):
    data['input'].append(expanded_data[i]['input'])
    data['target_text'].append(expanded_data[i]['target_text'])
    print(i)
    

785
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
27

In [ ]:
print(type(data['input'][0]))

In [15]:
for text in data['target_text']:
    l = 0
    if type(text) == list:
        l = l+1
        print (text)
        print(l)
print(l)

0


In [16]:
data["input"] = [
    text[0] if isinstance(text, list) else text for text in data["input"]
]

In [17]:
data["target_text"] = [
    text[0] if isinstance(text, list) else text for text in data["target_text"]
]

In [18]:
for text in data['input']:
    l = 0
    if type(text) == list:
        print (text)

In [ ]:
print(type(data['target_text']))

In [19]:
dataset = Dataset.from_dict(data)

In [20]:
dataset = dataset.train_test_split(test_size = 0.2, seed =32)

In [21]:
test_valid_split = dataset['test'].train_test_split(test_size = 0.5, seed = 32)

In [22]:
dataset = DatasetDict({
    'train' : dataset['train'],
    'validation': test_valid_split['train'],
    'test' : test_valid_split['test']
})

In [23]:
#tokenize data 
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [24]:
def preprocess_data(examples):
    inputs = tokenizer(examples["input"], max_length=512, truncation=True, padding="max_length")
    targets = tokenizer(examples["target_text"], max_length=512, truncation=True, padding="max_length").input_ids
    inputs["labels"] = targets
    return inputs

In [25]:
# Apply tokenization
tokenized_dataset = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/1256 [00:00<?, ? examples/s]

Map:   0%|          | 0/157 [00:00<?, ? examples/s]

Map:   0%|          | 0/157 [00:00<?, ? examples/s]

In [26]:
pip install absl-py rouge_score nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 103.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=4146ad6473aaa260e30b672663ffc3bd501a06ed3ada9f41b893f321b22d4173
  Stored in directory: /home/jovyan/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [27]:
#load the evaluation metric: ROUGE USED TO COPARE THE MACHINE GENERATED TEXT TO ONE OR MORE REFERENCE TEXTS CREATED BY HUMANS 
#used to measure the quality og machine translation and automatic summarization software
rouge = evaluate.load('rouge')

ROUGE scores provide an evaluation of how similar the generated text is to the reference text, which serves as a measure of quality rather than strict accuracy.

In [28]:
#computing the ROUGE score 
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Compute ROUGE scores
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {key: value for key, value in result.items()}  # Convert to percentage
    
    # Print results for each epoch
    print(f"Epoch evaluation result: {result}")
    return result

In [29]:
# Initialize model and training arguments
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
training_args = Seq2SeqTrainingArguments(
    output_dir="flan-t5-cwl-finetuned",
    eval_strategy="epoch",
    learning_rate=0.01, #edit this if necessary 
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01, #edit this if necassary 
    save_total_limit=3,
    num_train_epochs=30, # edit this if necessary 
    predict_with_generate=True
)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [30]:
# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    processing_class=tokenizer,
    compute_metrics=compute_metrics  # Use the custom compute_metrics function
)


In [31]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,2.954929,0.018890,0.011909,0.018653,0.018731
2,3.805600,2.511302,0.033292,0.023663,0.032907,0.032980
3,3.805600,2.278107,0.026674,0.011755,0.026355,0.026397
4,2.634300,2.084754,0.033292,0.023663,0.032907,0.032980
5,2.573800,2.169274,0.074329,0.050849,0.071153,0.071164
6,2.573800,2.139285,0.029456,0.002396,0.028508,0.028468
7,2.472400,2.316790,0.073952,0.049774,0.071080,0.070999
8,2.545800,2.237671,0.054851,0.039126,0.054141,0.054044
9,2.545800,2.537830,0.012047,0.005262,0.011948,0.012014
10,2.775100,2.610322,0.012859,0.005913,0.012721,0.012782


/opt/conda/lib/python3.11/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch evaluation result: {'rouge1': 0.0188896365913186, 'rouge2': 0.011908828696967582, 'rougeL': 0.018653245485828845, 'rougeLsum': 0.01873130935458895}


/opt/conda/lib/python3.11/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


...

/opt/conda/lib/python3.11/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch evaluation result: {'rouge1': 0.020877568865525564, 'rouge2': 0.009720349972548791, 'rougeL': 0.020326511179238317, 'rougeLsum': 0.0204759158533837}


/opt/conda/lib/python3.11/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch evaluation result: {'rouge1': 0.03500184130159931, 'rouge2': 0.013543683017902798, 'rougeL': 0.03209976114509412, 'rougeLsum': 0.03224717125470521}


TrainOutput(global_step=9420, training_loss=2.8169243166654225, metrics={'train_runtime': 25829.8704, 'train_samples_per_second': 1.459, 'train_steps_per_second': 0.365, 'total_flos': 2.580165704024064e+16, 'train_loss': 2.8169243166654225, 'epoch': 30.0})

In [ ]:
# Save the model and tokenizer
model.save_pretrained("flan-t5-cwl-finetuned")
tokenizer.save_pretrained("flan-t5-cwl-finetuned")